In [5]:
import oracledb
print(oracledb.__version__)

3.1.1


In [1]:
import oracledb

# Oracle connection details
username = "demo_user"
password = "demouser"
dsn = "localhost:1521/XEPDB1"  

# Establish connection and create cursor
try:
    # Connect to the Oracle database
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful")

    # Create a cursor
    cursor = connection.cursor()

except oracledb.DatabaseError as e:
    print("Database connection failed:", e)

Connection successful


In [5]:
try:
    # Create Banks Table
    cursor.execute("""
        CREATE TABLE Banks (
            bank_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
            bank_name VARCHAR2(100) NOT NULL,
            bank_location VARCHAR2(100),
            established_year NUMBER
        )
    """)

    # Create Reviews Table
    cursor.execute("""
        CREATE TABLE Reviews (
            review_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
            bank_id NUMBER,
            review_text CLOB,
            rating NUMBER CHECK (rating >= 1 AND rating <= 5),
            review_date DATE DEFAULT SYSDATE,
            FOREIGN KEY (bank_id) REFERENCES Banks(bank_id)
        )
    """)

    connection.commit()
    print("Tables created successfully.")
except oracledb.DatabaseError as e:
    print("Error creating tables:", e)


Tables created successfully.


In [6]:
# 2. Insert Banks
try:
    banks = [
        ("Commercial Bank of Ethiopia", None, None),
        ("Bank of Abyssinia", None, None),
        ("Dashen Bank", None, None)
    ]
    insert_bank_sql = "INSERT INTO Banks (bank_name, bank_location, established_year) VALUES (:1, :2, :3)"
    cursor.executemany(insert_bank_sql, banks)
    connection.commit()
    print(f"Inserted {len(banks)} banks into 'Banks' table.")
except oracledb.DatabaseError as e:
    print("Error inserting banks:", e)

Inserted 3 banks into 'Banks' table.


In [7]:
# Fetch inserted banks
cursor.execute("SELECT bank_id, bank_name FROM Banks")
bank_rows = cursor.fetchall()
print("Banks in database:")
for bank_id, name in bank_rows:
    print(f"Bank ID: {bank_id}, Name: {name}")


Banks in database:
Bank ID: 1, Name: Commercial Bank of Ethiopia
Bank ID: 2, Name: Bank of Abyssinia
Bank ID: 3, Name: Dashen Bank


In [8]:
import pandas as pd

# Load the cleaned reviews CSV
df = pd.read_csv('data/cleaned_bank_reviews.csv')


In [ ]:
# Fetch bank IDs to map bank names to bank_id
cursor.execute("SELECT bank_id, bank_name FROM Banks")
bank_id_map = {name: bank_id for bank_id, name in cursor.fetchall()}


In [ ]:
# Map the bank names in your dataframe to their corresponding bank_id

reviews_to_insert = []

for _, row in df.iterrows():
    bank_id = bank_id_map.get(row['bank'])
    if bank_id:
        # Prepare tuple: (bank_id, review_text, rating, review_date)
        reviews_to_insert.append((
            bank_id,
            row['review'],
            int(row['rating']),
            row['date']  
        ))


In [ ]:
# Insert reviews using executemany()

insert_review_sql = """
    INSERT INTO Reviews (bank_id, review_text, rating, review_date)
    VALUES (:1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'))
"""

try:
    cursor.executemany(insert_review_sql, reviews_to_insert)
    connection.commit()
    print(f"Inserted {len(reviews_to_insert)} reviews successfully.")
except oracledb.DatabaseError as e:
    print("Error inserting reviews:", e)


Inserted 1200 reviews successfully.


In [ ]:
# Verify insertion
cursor.execute("SELECT COUNT(*) FROM Reviews")
count = cursor.fetchone()[0]
print(f"Total reviews in database: {count}")


Total reviews in database: 1200
